In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from vis_tools import grab_gmt_features, read_hysplit_netcdf

##############################################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18040212_taupo_3.0_0.01.nc'

# SPECIFY: pick one of 8 time steps (indices 0-7, 3 hr chunks)
TIME_STEP = 0

# SPECIFY: plotting params
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
CONTOURS = None  # number of contours (or specify None)
RES = 'f'  # detail of GMT features ('c', 'l', 'i', 'h', 'f')
##############################################################

model = read_hysplit_netcdf(FILENAME)

time_slice = model['total_deposition'].values[TIME_STEP, :, :]  # grab time step from the gridded data        
np.place(time_slice, time_slice==0, np.nan)  # replace all zero values with nan's

# tweak figure settings for readability
plt.figure(figsize=(9, 6))
mpl.rcParams.update({'font.size': 12})

# PLOT (projected) background geography
features = grab_gmt_features(-48, -34, 166, 180, RES)  # this includes all of NZ
for f_lat, f_lon in zip(features['latitude'], features['longitude']):
    plt.plot(f_lon, f_lat, 'k', linewidth=0.75, zorder=3)

# PLOT ashfall (using log scale)
model_lat =  model['lat'].values
model_lon = model['lon'].values
plt.pcolor(model_lon, model_lat, time_slice, cmap='inferno_r', norm=colors.LogNorm(), vmin=ASH_MIN, vmax=ASH_MAX, zorder=1)
cb = plt.colorbar()

# PLOT source location
src_lat = model.attrs['volcano_location'][0]
src_lon = model.attrs['volcano_location'][1]
plt.scatter(src_lon, src_lat, s=100, c='c', marker='^', edgecolors='k', linewidths=0.75, zorder=4, label='source')

# HARD-CODED extent for now
plt.xlim(175, 178)
plt.ylim(-40, -38)
plt.gca().set_aspect('equal', adjustable='box')

plt.xlabel('lon')
plt.ylabel('lat')
plt.title('UNPROJECTED\n' + FILENAME.split('/')[-1] + '\n' + str((TIME_STEP+1)*3) + ' hrs post-event')
cb.set_label('ash thickness (mm)')

plt.legend();